### Import Libraries

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import DataLoader, Dataset
from torch.nn import Sequential, Linear, ReLU, Sigmoid, BCELoss
from torch.optim import SGD
from sklearn.metrics import accuracy_score

### Import Data

In [2]:
dataset = pd.read_csv('Churn.csv')

Format the data

In [3]:
features = pd.get_dummies(dataset.drop(["Churn", "Customer ID"], axis=1))
labels = dataset["Churn"].apply(lambda x: 1 if x == "Yes" else 0)

featureTrain, featureTest, labelTrain, labelTest = train_test_split(features, labels, test_size=0.2)

featureTrain = torch.tensor(featureTrain.to_numpy().astype(float))
featureTest = torch.tensor(featureTest.to_numpy().astype(float))
labelTrain = torch.tensor(labelTrain.to_numpy().astype(float))
labelTest = torch.tensor(labelTest.to_numpy().astype(float))

class ChurnDataset(Dataset):
    def __init__(self, dataset):
        self.x_data = featureTrain
        self.y_data = labelTrain
        
    def __len__(self):
        return len(self.x_data)
    
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

trainingSet = DataLoader(ChurnDataset(dataset), batch_size=32, shuffle=True)

### Make Model

In [4]:
class ANN(torch.nn.Module):
    def __init__(self):
        super().__init__()

        # Create model (32, 64, 1)
        self.model = Sequential(
            Linear(6575, 32),
            ReLU(),

            Linear(32, 64),
            ReLU(),

            Linear(64, 1),
            Sigmoid()
        )

        # Set optimizer as SGD
        self.optimizer = SGD(self.parameters(), lr=0.0001)

        # Set loss function as binary cross entropy
        self.loss = BCELoss()
    
    def forward(self, x):
        return self.model(x)

In [5]:
model = ANN().to("cuda")

# Train model
for epoch in range(200):
    for data, label in trainingSet:
        data = data.to("cuda").to(torch.float32)
        label = label.to("cuda").to(torch.float32)

        # Forward pass
        output = model(data) 

        # Calculate loss
        loss = model.loss(output, label.unsqueeze(1))

        # Zero gradients
        model.optimizer.zero_grad()

        # Backward pass
        loss.backward()

        # Update weights
        model.optimizer.step() 
    
    print(f"Epoch: {epoch+1}, Loss: {loss.item()}")

Epoch: 1, Loss: 0.49798497557640076
Epoch: 2, Loss: 0.4121250510215759
Epoch: 3, Loss: 0.3391192853450775
Epoch: 4, Loss: 0.8715300559997559
Epoch: 5, Loss: 0.6026017069816589
Epoch: 6, Loss: 0.32133039832115173
Epoch: 7, Loss: 0.7252461314201355
Epoch: 8, Loss: 0.2249997854232788
Epoch: 9, Loss: 0.6199973225593567
Epoch: 10, Loss: 0.6163090467453003
Epoch: 11, Loss: 0.7888840436935425
Epoch: 12, Loss: 0.42025303840637207
Epoch: 13, Loss: 0.8173975348472595
Epoch: 14, Loss: 0.33355212211608887
Epoch: 15, Loss: 0.7276594638824463
Epoch: 16, Loss: 0.4970651865005493
Epoch: 17, Loss: 0.27258872985839844
Epoch: 18, Loss: 0.5820369720458984
Epoch: 19, Loss: 0.3173601031303406
Epoch: 20, Loss: 0.5132920742034912
Epoch: 21, Loss: 0.6749348640441895
Epoch: 22, Loss: 0.43367230892181396
Epoch: 23, Loss: 0.371406614780426
Epoch: 24, Loss: 0.7812720537185669
Epoch: 25, Loss: 0.39083385467529297
Epoch: 26, Loss: 0.23620931804180145
Epoch: 27, Loss: 0.27856674790382385
Epoch: 28, Loss: 0.7421338558

In [6]:
y_pred = model(featureTest.to("cuda").to(torch.float32))
y_pred = [1 if x >= 0.5 else 0 for x in y_pred]

In [7]:
accuracy_score(labelTest, y_pred)

0.8005677785663591